In [1]:
import csv
import sqlite3

def read_csv_data(csv_file_path):
    """Read data from CSV file and return as list of dictionaries"""
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return [row for row in reader]

def initialize_database(db_path, data):
    """Create database and table with appropriate columns"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    columns = data[0].keys()
    columns_with_types = ", ".join([f"{col} TEXT" for col in columns])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS data_table ({columns_with_types})")
    
    return conn, cursor

def insert_data(cursor, data):
    """Insert data into the SQLite database"""
    for row in data:
        placeholders = ", ".join(["?"] * len(row))
        values = tuple(row.values())
        cursor.execute(f"INSERT INTO data_table VALUES ({placeholders})", values)

def main():
    csv_file_path = "./database/Pull_Request_Data.csv"
    db_path = "./database/Pull_request_Data_SQLITE.db"
    
    data = read_csv_data(csv_file_path)
    conn, cursor = initialize_database(db_path, data)
    insert_data(cursor, data)
    
    conn.commit()
    conn.close()
    print("Data successfully moved from CSV file to SQLite database.")

if __name__ == "__main__":
    main()

Data successfully moved from CSV file to SQLite database.


In [2]:
def add_merge_time_column(db_path):
    """Add and calculate merge time column"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Add new column
    cursor.execute("ALTER TABLE data_table ADD COLUMN merge_Ti TEXT")
    
    # Calculate merge time
    cursor.execute("""
        UPDATE data_table
        SET merge_Ti = 
            CASE 
                WHEN Merged_Date IS NOT NULL AND Merged_Date != '' 
                THEN ( julianday(Merged_Date) - julianday(Created_Date)) || ' days'
                ELSE NULL
            END
    """)
    
    conn.commit()
    conn.close()
    print("Column 'merge_time' added and updated successfully.")

# Execute the function
add_merge_time_column("./database/Pull_request_Data_SQLITE.db")

Column 'merge_time' added and updated successfully.


In [3]:
def inspect_database(db_path):
    """Inspect database structure and content"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Show tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:", tables)
    
    # Show schema
    cursor.execute("PRAGMA table_info(data_table);")
    schema = cursor.fetchall()
    print("\nSchema of the table:")
    for column in schema:
        print(column)
    
    # Show sample data
    cursor.execute("SELECT * FROM data_table LIMIT 10;")
    rows = cursor.fetchall()
    print("\nFirst 10 rows in the table:")
    for row in rows:
        print(row)
    
    conn.close()

# Execute the function
inspect_database("./database/Pull_request_Data_SQLITE.db")

Tables in the database: [('data_table',)]

Schema of the table:
(0, 'PR_ID', 'TEXT', 0, None, 0)
(1, 'Created_Date', 'TEXT', 0, None, 0)
(2, 'Merged_Date', 'TEXT', 0, None, 0)
(3, 'Assignee', 'TEXT', 0, None, 0)
(4, 'Status', 'TEXT', 0, None, 0)
(5, 'Reviewer_Feedback', 'TEXT', 0, None, 0)
(6, 'merge_Ti', 'TEXT', 0, None, 0)

First 10 rows in the table:
('1', '2024-12-17 07:15:08', '2024-12-21 07:15:08', 'Bob', 'merged', 'Pending review', '4.0 days')
('2', '2025-01-12 07:15:08', '2025-01-17 07:15:08', 'Eve', 'closed', 'Good', '5.0 days')
('3', '2025-01-21 07:15:08', '', 'Alice', 'open', 'Good', None)
('4', '2025-01-21 07:15:08', '2025-01-20 07:15:08', 'Eve', 'merged', 'Good', '-1.0 days')
('5', '2025-01-14 07:15:08', '', 'David', 'open', 'Needs changes', None)
('6', '2024-12-20 07:15:08', '2024-12-29 07:15:08', 'David', 'merged', 'Needs changes', '9.0 days')
('7', '2025-01-05 07:15:08', '2025-01-14 07:15:08', 'Charlie', 'merged', 'Needs changes', '9.0 days')
('8', '2025-01-20 07:15:08'